# HW4
CSCI 4930  
Justin Hoang  

## Reading the Dataset

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('spiral-dataset.csv', index_col=None, sep=r'\s', header=None)
data.columns = ['x', 'y', 'target']

data.head()

/var/folders/pt/_3pv06z57s3_j0w3tq9l60p80000gn/T/ipykernel_22606/3642716612.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('spiral-dataset.csv', index_col=None, sep=r'\s', header=None)


,x,y,target
0,31.95,7.95,3
1,31.15,7.30,3
2,30.45,6.65,3
3,29.70,6.00,3
4,28.90,5.55,3


In [1]:
import numpy as np

### Calculate Euclidean Distance Function

In [3]:
# euclidean distance - sqrt of sum squared
def euclidean_distance(p1, p2):
    return np.sqrt(np.sum(p1 - p2) ** 2)

### Create Distance Matrix Function

In [4]:
# distance matrix
def distance_matrix(data):
    n = len(data)
    
    dist_matrix = np.zeros((n,n))
    
    for i in range(n):
        for k in range(n):
            dist = euclidean_distance(data[i], data[k])
            dist_matrix[i, k] = dist
            dist_matrix[k, i] = dist
            
    return dist_matrix

### Plot Data Function

In [5]:
def plot_data(name, clusters):
    cluster_data_x = [[], [], []]
    cluster_data_y = [[], [], []]
    
    # data points
    for i in clusters[0]:
        data = dataset[i]
        cluster_data_x[0].append(data[0])
        cluster_data_y[0].append(data[1])
        
    for i in clusters[1]:
        data = dataset[i]
        cluster_data_x[1].append(data[0])
        cluster_data_y[1].append(data[1])
        
    for i in clusters[2]:
        data = dataset[i]
        cluster_data_x[2].append(data[0])
        cluster_data_y[2].append(data[1])
        
    return cluster_data_x, cluster_data_y

### Create Linkage Clustering Function
